In [ ]:
!python --version
!pip install tqdm
!pip install JPype1-0.7.1-cp37-cp37m-win_amd64.whl

In [ ]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다.
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm import tqdm

### 분석 키워드 설정

In [ ]:
List = [#'키워드 설정']

In [ ]:
for query_txt in List:
# query_txt2 = input('2.제외할 첫번째 키워드를 입력하세요: ')
# query_txt3 = input('3.제외할 두번째 키워드를 입력하세요: '|)
    print (query_txt)
start_date = 20190101 #input('4.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :')
end_date = 20200801 #input('5.조회를 종료할 날짜를 입력하세요(예:2017-12-31): ')
print(start_date)
print(end_date)

### 키워드 제목, URL 크롤링

In [ ]:
for query_txt in List:
# query_txt2 = input('2.제외할 첫번째 키워드를 입력하세요: ')
# query_txt3 = input('3.제외할 두번째 키워드를 입력하세요: '|)
    print (query_txt)
    start_date = "20190101" #input('4.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :')
    end_date = "20200801" #input('5.조회를 종료할 날짜를 입력하세요(예:2017-12-31): ')
#     print(start_date)
#     print(end_date)
#Step 1. 크롬 웹브라우저 실행
    path = "chromedriver.exe"

    driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe')
# 사이트 주소
    driver.get('http://www.naver.com')
    time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
    element = driver.find_element_by_id("query")
    element.send_keys(query_txt) 
    element.submit()

#Step 3. "블로그" 카테고리 선택
    driver.find_element_by_link_text("블로그").click( )    # .click() 괄호 안을 눌러라는 뜻

#Step 4. 오른쪽의 검색 옵션 버튼 클릭
    driver.find_element_by_id("_search_option_btn").click( )

#Step 5. 정렬 : "관련도순" 
    driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click( )  # 정렬 버튼의 xpath 클릭
    driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click( ) # 관련도순 xpath

#Step 6. 날짜 입력
    driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
    time.sleep(2)

# 시작 날짜 입력하기
    s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
    driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
    s_date.clear( )  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
    time.sleep(1)
# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력하는 부분입니다.
    for c in start_date:
        s_date.send_keys(c)
        time.sleep(0.1)
    
# 종료 날짜 입력하기
    e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
    driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
    e_date.clear()
    time.sleep(1)

    for c in end_date:
        e_date.send_keys(c)
        time.sleep(0.1)
    
# Step7. 날짜 입력 "적용하기" 버튼을 클릭 합니다.  
    driver.find_element_by_class_name("tx").click()
    time.sleep(1)

# # Step 8. 상세 검색버튼을 클릭 후 제외할 단어들을 설정합니다.
# ele2 = driver.find_element_by_id("inpop3")
# ele2.send_keys(query_txt2)
# ele2.send_keys(',')
# ele2.send_keys(query_txt3)
# driver.find_element_by_css_selector(".btn_ft.ty_green._search").click( )
    url_list = []
    title_list = []

# 몇개의 페이지를 크롤링할지 선택
    total_page = 10
    for i in tqdm(range(0, total_page)):  # 페이지 번호
        i = i*10 + 1
        url = "https://search.naver.com/search.naver\
?date_from={0}&date_option=8&date_to={1}\
&dup_remove=1&nso=p%3Afrom{2}to{3}post_blogurl=\
&post_blogurl_without=&query={4}&sm=tab_pge&srchby=all&st=sim&where=post&start={5}".format(start_date,end_date,start_date,end_date,query_txt, i)
        driver.get(url)
        time.sleep(0.5)
    
    # URL 크롤링 시작
        titles = "a.sh_blog_title._sp_each_url._sp_each_title"
        article_raw = driver.find_elements_by_css_selector(titles)
#     article_raw

    # url 크롤링 시작    
        for article in article_raw:
            url = article.get_attribute('href')   
            url_list.append(url)
    
    # 제목 크롤링 시작    
        for article in article_raw:
            title = article.get_attribute('title')   
            title_list.append(title)
    
            print(title)
    
    print('url갯수: ', len(url_list))
    print('url갯수: ', len(title_list))

    df = pd.DataFrame({'url':url_list, 'title':title_list})

# 저장하기
    df.to_excel("blog_url_{0}.xlsx".format(query_txt))

### 블로그 본문 크롤링

In [ ]:
# "url_list.csv" 불러오기
for query_txt in List:
    url_load = pd.read_excel("blog_url_{0}.xlsx".format(query_txt))        # 기본 모델

#     num_list = len(url_load)

#     print(num_list)
#     url_load.head()
    dict = {}  # 전체 크롤링 데이터를 담을 그릇

# ★수집할 글 갯수
    number = 100
    for i in tqdm(range(0, number)): 
    # 글 띄우기
        url = url_load['url'][i]
        driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe')
        driver.get(url)   # 글 띄우기
    
    # 크롤링
    
        try : 
            # iframe 접근
            driver.switch_to_frame('mainFrame')

            target_info = {}

            # 제목 크롤링 시작
            overlays = ".se-fs-.se-ff-"                                 
            tit = driver.find_element_by_css_selector(overlays)         # title
            title = tit.text
            title

#         # 글쓴이 크롤링 시작
#         overlays = ".nick"                                 
#         nick = driver.find_element_by_css_selector(overlays)         # nick
#         nickname = nick.text

            # 날짜 크롤링
            overlays = ".se_publishDate.pcol2"                                 
            date = driver.find_element_by_css_selector(overlays)         # date
            datetime = date.text

            # 내용 크롤링
            overlays = ".se-component.se-text.se-l-default"                                 
            contents = driver.find_elements_by_css_selector(overlays)         # date

            content_list = []
            for content in contents:
                content_list.append(content.text)

            content_str = ' '.join(content_list)

            # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
            target_info['title'] = title
    #         target_info['nickname'] = nickname
            target_info['datetime'] = datetime
            target_info['content'] = content_str

            # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
            dict[i] = target_info
            time.sleep(1)
        
            print(i, title)

            # 글 하나 크롤링 후 크롬 창 닫기
            driver.close()       
    
        # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동
        except:
            driver.close()
            time.sleep(1)
            continue
    
        # 중간,중간에 파일로 저장하기
        if i % 20 == 0 :
            # 판다스로 만들기
            import pandas as pd
            result_df = pd.DataFrame.from_dict(dict, 'index')

            # 저장하기
            result_df.to_excel("blog_content_{0}.xlsx".format(query_txt))
            time.sleep(3)
    
    print('수집한 글 갯수: ', len(dict))
    # print(dict)    
    
    # 판다스로 만들기
    result_df = pd.DataFrame.from_dict(dict, 'index')

    # 저장하기
    result_df.to_excel("blog_content_{0}.xlsx".format(query_txt))

### 판다스 통합

In [ ]:
temp_data1 = pd.read_excel("blog_content_동두천자연휴양림.xlsx")
temp_data2 = pd.read_excel("blog_content_한탄강주상절리길.xlsx")
temp_data3 = pd.read_excel("blog_content_여강길.xlsx")
temp_data4 = pd.read_excel("blog_content_잣향기푸른숲.xlsx")
temp_data5 = pd.read_excel("blog_content_바람새마을 소풍정원.xlsx")
temp_data6 = pd.read_excel("blog_content_평화누리공원.xlsx")
temp_data7 = pd.read_excel("blog_content_행주산성역사공원.xlsx")
temp_data8 = pd.read_excel("blog_content_갯골생태공원.xlsx")
temp_data9 = pd.read_excel("blog_content_김포 평화누리길 1코스.xlsx")
temp_data10 = pd.read_excel("blog_content_곤지암.xlsx")

temp_data1.head()

### 내용 텍스트 분석

In [ ]:
!pip install graphviz
!pip install konlpy

import scipy as sp
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 그래프
%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz             
from sklearn.tree import export_graphviz

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)

# 워닝 없애주는 것
import warnings  
warnings.filterwarnings('ignore')

### 형태소 분석

In [ ]:
#형태소 분석
from konlpy.tag import Kkma       ; kkma = Kkma()
from konlpy.tag import Hannanum   ; hannanum = Hannanum()
from konlpy.tag import Okt        ; t = Okt()
from konlpy.tag import *
from konlpy.tag import Twitter
import pickle
import nltk

In [ ]:
for query_txt in List:
    raw = pd.read_excel("blog_content_{0}.xlsx".format(query_txt)) 
    content_list = raw.content.values.tolist()
    content_list
    print(len(content_list))

In [ ]:
for query_txt in List:
    raw = pd.read_excel("blog_content_{0}.xlsx".format(query_txt)) 
    content_list = raw.content.values.tolist()

    print(len(content_list))
    content_list
    # 리스트 중 str 타입이 아닌 요소들이 존재함을 확인
    for i in content_list:
        if type(i) == float:
            i = str(i)
#             print(i)
    # str 타입이 아닌 요소들이 있으면 전부 str 타입으로 바꿔라!
    for i in range(len(content_list)):
        if type(content_list[i]) != str:
            content_list[i] = str(content_list[i])
            
    content_text = ''

    for each_line in content_list[:2000]:
        content_text = content_text + each_line + '\n'
    print(content_text)
    
    tokens_ko = t.morphs(content_text)
    tokens_ko

In [ ]:
for query_txt in List:
    raw = pd.read_excel("blog_content_{0}.xlsx".format(query_txt)) 
    content_list = raw.content.values.tolist()

    print(len(content_list))
    content_list
    # 리스트 중 str 타입이 아닌 요소들이 존재함을 확인
    for i in content_list:
        if type(i) == float:
            i = str(i)
            print(i)
    # str 타입이 아닌 요소들이 있으면 전부 str 타입으로 바꿔라!
    for i in range(len(content_list)):
        if type(content_list[i]) != str:
            content_list[i] = str(content_list[i])
            
    content_text = ''

    for each_line in content_list[:2000]:
        content_text = content_text + each_line + '\n'
    


#     twitter = Twitter()
#     raw_pos_tagged = twitter.pos(content_text, norm=True, stem=True) # POS Tagging
#     raw_pos_tagged

In [ ]:
tokens_ko = t.morphs(content_text)

In [ ]:
# 전체 갯수 확인
ko = nltk.Text(tokens_ko)   
print(len(ko.tokens))          # 토큰 전체 갯수
print(len(set(ko.tokens)))     # 토큰 unique 갯수

In [ ]:
ko = nltk.Text(tokens_ko)
ko.vocab().most_common(100)    # 가장 많이 나온 단어 100개

In [ ]:
#불용어 제거
# 불용어 : 인터넷 검색 시 검색 용어로 사용하지 않는 단어. 관사, 전치사, 조사, 접속사 등 검색 색인 단어로 의미가 없는 단어
stop_words = [')','?','1','"(', '_', ')/','\n','.',',', '<','!','(','(', '??','..', '4', '|', '>', '?(', '"…', '#', '&', '・', "']",'.',' ','/',"'",'’','”','“','·', '[','!','\n','·','‘','"','\n ',']',':','…',')','(','-', 'nan','가','요','답변','...','을','수','에','질문','제','를','이','도',
                      '좋','1','는','로','으로','2','것','은','다',',','니다','대','들',
              '이다','하고','입니다','대한','에서','수사','심의',
                      '2017','들','데','..','의','때','겠','고','게','네요','한','일','할',
                      '10','?','하는','06','주','려고','인데','거','좀','는데','~','ㅎㅎ',
                      '하나','이상','20','뭐','까','있는','잘','습니다','다면','했','주려',
                      '지','있','못','후','중','줄','6','과','어떤','기본','!!',
                      '단어','라고','중요한','합','가요','....','보이','네','무지',
             '적', '성', '삼', '등', '전', '인', '그', '했다', '와', '위', '해', '권', '된', '서', '말', '분']

tokens_ko = [each_word for each_word in tokens_ko
             if each_word not in stop_words]

ko = nltk.Text(tokens_ko)
ko.vocab().most_common(50)

### 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from sklearn.tree import export_graphviz
plt.figure(figsize=(15,6))
ko.plot(50) 
plt.show()

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)

In [ ]:
data = ko.vocab().most_common(300)

print(len(data))
data

In [ ]:
# list tuple을 딕셔너리로 만들어주는 함수
def todict(list_tuple):    
    todict = {}
    for i in range(0,len(list_tuple)):
        todict[data[i][0]] = data[i][1]
    return todict

In [ ]:
# 워드클라우드를 그려보자
wordcloud = WordCloud(font_path='c:/Windows/Fonts/malgun.ttf',
                      relative_scaling = 0.2,
                      #stopwords=STOPWORDS,
                      background_color='white',
                      ).generate_from_frequencies(todict(data))

plt.figure(figsize=(16,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()